In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

# === 1. 시뮬레이션 파라미터 설정 ===
n_samples = 100
dt = 0.01
total_time = 2.0
n_steps = int(total_time / dt)

# 저장할 시간 간격 설정
plot_interval = 0.2
steps_per_plot = int(plot_interval / dt)

# 초기 위치
x0 = np.zeros((n_samples, 2))

# === 2. 확률 과정 모델 파라미터 ===
drift_fwd = np.array([1, 0])
drift_bwd = np.array([-1, 0])
diffusion_matrix = np.array([[1/2, 0], [0, 1/4]])
sigma = np.sqrt(diffusion_matrix)

# === 3. 이미지 저장을 위한 폴더 생성 ===
output_dir = "simulation_steps"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"'{output_dir}' 폴더를 생성했습니다.")

# === 4. 시뮬레이션 및 이미지 생성 ===

current_positions = np.copy(x0)
plot_count = 0

# 전체 시뮬레이션의 X, Y 최대/최소 범위를 대략적으로 계산하여 축을 고정
# (매번 그림의 축이 바뀌는 것을 방지하여 움직임을 잘 보이게 함)
axis_limit_x = (-4, 5)
axis_limit_y = (-3.5, 3.5)

def save_plot(positions, time_point, count):
    """현재 입자 위치를 이미지 파일로 저장하는 함수"""
    plt.figure(figsize=(8, 7))
    plt.scatter(positions[:, 0], positions[:, 1], alpha=0.7, c='blue')
    plt.scatter([0],[0], alpha=0.7, c='red')
    plt.title(f"Particle Distribution at t = {time_point:.1f}s", fontsize=16)
    plt.xlabel("X coordinate")
    plt.ylabel("Y coordinate")
    
    # 모든 이미지의 축 범위를 동일하게 설정
    plt.xlim(axis_limit_x)
    plt.ylim(axis_limit_y)
    
    plt.grid(True, linestyle=':', alpha=0.6)
    
    # 파일명 설정 (예: step_00.png, step_01.png, ...)
    filename = os.path.join(output_dir, f"step_{count:02d}.png")
    plt.savefig(filename)
    plt.close() # 메모리 누수 방지를 위해 창을 닫음

# 시작 상태 (t=0) 저장
save_plot(current_positions, 0.0, plot_count)
plot_count += 1

# 시뮬레이션 루프
for i in range(n_steps):
    current_time = (i + 1) * dt
    
    # 드리프트 벡터 선택
    drift = drift_fwd if current_time <= 1.0 else drift_bwd
    
    # 위치 업데이트
    dW = np.sqrt(dt) * np.random.randn(n_samples, 2)
    update = drift * dt + np.dot(dW, sigma.T)
    current_positions += update

    # 저장 간격이 되면 이미지 저장
    if (i + 1) % steps_per_plot == 0:
        save_plot(current_positions, current_time, plot_count)
        plot_count += 1